# CNN model

In [1]:
# from pylsl import StreamInlet, resolve_stream
# import numpy as np
# import mne
# import pywt
# import pickle
# from keras.models import load_model
# import pandas as pd
# from datetime import datetime
# import os
# from sklearn.preprocessing import StandardScaler
# from ssqueezepy import ssq_cwt, ssq_stft
# import socket
# sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
# serverAddressPort = ("127.0.0.1", 1880)
# # import warnings
# # warnings.simplefilter(action='ignore', category=FutureWarning)

# pretrained_model = load_model("CNN+RNN_model_newsess_nonvsim_slide_weights.h5")

# with open('trained_csp_newsess_nonvsim_slide_model.pkl', 'rb') as file:
#     trained_csp = pickle.load(file)

# data_size = 501
# channel_count = 8

# epoch_2s = np.zeros(shape=(8,data_size))

# # first resolve an EEG stream on the lab network
# print("looking for an EEG stream...")
# All_streams = resolve_stream()
# print(All_streams)

# for i in All_streams:
#     print(i.name())

# unicorn_stream =[stream for stream in All_streams if stream.name() == 'openvibefilter']

# data = []
# time_count = 0.0
# time_all = 0.0
# time_interval = 0.1

# inlet = StreamInlet(unicorn_stream[0])

# fs = 250.0
 
# channels = ["Fz","C3", "Cz", "C4","Pz","PO7","Oz","PO8"]

# info = mne.create_info(
#     ch_names= channels,
#     ch_types= ['eeg']*len(channels),
#     sfreq= fs
# )

# csv_file = 'real_time_data.csv'
# if os.path.exists(csv_file):
#     open(csv_file, 'w').close()

# def to_excel(y_pred_raw):
#     new_data_point = {
#     'Timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
#     'Class_Raw': y_pred_raw,
#     'Class': np.argmax(y_pred_raw),
#     }
#     df = pd.DataFrame([new_data_point])
#     try:
#         df.to_csv(csv_file, mode='x', index=False)
#     except FileExistsError:
#         df.to_csv(csv_file, mode='a', header=False, index=False)


# def call_pywt(new_data):
#     component_num = channel_count
#     train_cwt = np.ndarray(shape=(new_data.shape[0], new_data.shape[2], component_num))
#     for jj in range(0, new_data.shape[0]):
#         train_cwt[jj] = new_data[jj].T

#     scales = range(1,31)
#     waveletname = 'morl'
#     train_size = new_data.shape[0]
#     train_data_cwt = np.ndarray(shape=(train_size, len(scales), new_data.shape[2], component_num))

#     for ii in range(0,train_size):
#         for jj in range(0,component_num):
#             signal = train_cwt[ii, :, jj]
#             coeff, _ = pywt.cwt(signal, scales, waveletname, 1)
#             coeff_ = coeff[:,:new_data.shape[2]]  
#             train_data_cwt[ii, :, :, jj] = abs(coeff_)

#     train_cwt_stack = np.ndarray(shape=(train_size, len(scales)*component_num, new_data.shape[2]))

#     for jj in range(0,train_data_cwt.shape[0]):
#         train_cwt_stack[jj] = np.vstack((train_data_cwt[jj,:,:,0], train_data_cwt[jj,:,:,1], train_data_cwt[jj,:,:,2], train_data_cwt[jj,:,:,3], train_data_cwt[jj,:,:,4], train_data_cwt[jj,:,:,5], train_data_cwt[jj,:,:,6],train_data_cwt[jj,:,:,7] ))
#     return train_cwt_stack

# def call_stft(new_data):
#     component_num = 8
#     n_fft = 256  # Number of DFT points
#     hop_length = int(n_fft * 0.03)  # 97% overlapping
#     win_length = int(n_fft * 0.5)   # 0.5 seconds window length
#     window = 'hamming' 
#     train_size = new_data.shape[0]
#     train_data_stft = np.ndarray(shape=(train_size,component_num, 129, 72))

#     for i in range(0,train_size):
#         _,coeff, *_ = ssq_stft(new_data[i,:], n_fft=n_fft, hop_len=hop_length, win_len=win_length, window=window)
#         train_data_stft[i, :, :, :] = coeff

#     # Stack array and convert to image
#     train_stft_stack = np.ndarray(shape=(train_size, train_data_stft.shape[2],train_data_stft.shape[3]*component_num))

#     for jj in range(0,train_data_stft.shape[0]):
#         train_stft_stack[jj] = np.concatenate(train_data_stft[jj], axis = 1)
#     return train_stft_stack


# while True:

#     sample, timestamp = inlet.pull_sample()
#     if sample:
#         data1 = sample[0:8]
#         data.append(data1)  

#         time_count += 1 * 0.004
#         time_count = np.round(time_count ,3)
#         time_all += 1 * 0.004

#         data = np.asarray(data)
#         epoch_2s = np.delete(epoch_2s, slice(0,1), axis =1)
#         epoch_2s = np.append(epoch_2s, data.T, axis= 1)

#         data = np.ndarray.tolist(data)
#         data = []

#         if time_count == time_interval:
            
#             # data = np.asarray(data)
#             # epoch_2s = np.delete(epoch_2s, slice(0,int(time_interval/0.004)), axis =1)
#             # epoch_2s = np.append(epoch_2s, data.T, axis= 1)
            
#             raw = mne.io.RawArray(epoch_2s, info, verbose=False)
#             eeg1= raw.pick(["Fz","C3", "Cz", "C4","Pz",'PO7','Oz','PO8'])
#             mne_epoch = mne.EpochsArray(eeg1.get_data().reshape(1,channel_count,data_size), info, baseline=(0.0,0.0), verbose=False)
#             mne_epoch = mne_epoch.copy().crop(tmin=0.0, tmax=2.0)

#             new_data = trained_csp.transform(mne_epoch.get_data().reshape(1,channel_count,501))

#             train_cwt_stack = call_pywt(new_data)

#             # scaler = StandardScaler()
#             # scaled_data = scaler.fit_transform(train_cwt_stack[0])
#             # scaled_data = scaled_data.reshape(1,240,501)

#             y_pred_raw = pretrained_model.predict(train_cwt_stack ,verbose=False)
#             y_pred_raw = np.around(y_pred_raw,3)
            
#             print("predict class", np.argmax(y_pred_raw), datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

#             to_excel(y_pred_raw[0])

#             #Send to Unity
#             # send = str('{:.3f}'.format(y_pred_raw[0][0])) +','+ str('{:.3f}'.format(y_pred_raw[0][1]))+','+ str('{:.3f}'.format(y_pred_raw[0][2]))
#             # sock.sendto(str.encode(send), serverAddressPort)

#             # Reset
#             # data = np.ndarray.tolist(data)
#             # data = []
#             time_count = 0.0
        
#         # if time_all >= 300.0:
#         #     break

looking for an EEG stream...
[<pylsl.pylsl.StreamInfo object at 0x000001FD747E14D0>, <pylsl.pylsl.StreamInfo object at 0x000001FD74D2D150>]
openvibefilter
openvibeMarker
predict class 0 2024-01-19 16:52:28
predict class 0 2024-01-19 16:52:28
predict class 0 2024-01-19 16:52:28
predict class 0 2024-01-19 16:52:28
predict class 0 2024-01-19 16:52:28
predict class 0 2024-01-19 16:52:29
predict class 0 2024-01-19 16:52:29
predict class 0 2024-01-19 16:52:29
predict class 0 2024-01-19 16:52:29
predict class 0 2024-01-19 16:52:29
predict class 0 2024-01-19 16:52:29
predict class 0 2024-01-19 16:52:29
predict class 0 2024-01-19 16:52:29
predict class 0 2024-01-19 16:52:29
predict class 0 2024-01-19 16:52:29
predict class 0 2024-01-19 16:52:30
predict class 0 2024-01-19 16:52:30
predict class 0 2024-01-19 16:52:30
predict class 0 2024-01-19 16:52:30
predict class 0 2024-01-19 16:52:30
predict class 0 2024-01-19 16:52:30
predict class 0 2024-01-19 16:52:30
predict class 0 2024-01-19 16:52:30
pr

KeyboardInterrupt: 

# CSP+LDA

In [1]:
from pylsl import StreamInlet, resolve_stream
import numpy as np
import matplotlib.pyplot as plt
import mne
import pickle
from sklearn.metrics import confusion_matrix
import seaborn as sns
from matplotlib.animation import FuncAnimation
import os
from datetime import datetime
import pandas as pd

epoch_2s = np.zeros(shape=(8,751))

with open('trained_csp2_model.pkl', 'rb') as file:
    trained_csp2 = pickle.load(file)

with open('trained_lda_model.pkl', 'rb') as file:
    trained_lda = pickle.load(file)

# first resolve an EEG stream on the lab network
print("looking for an EEG stream...")
All_streams = resolve_stream()
print(All_streams)

for i in All_streams:
    print(i.name())

unicorn_stream =[stream for stream in All_streams if stream.name() == 'openvibefilter']

data = []
time_count = 0.0
time_all = 0.0
time_interval = 0.1

inlet = StreamInlet(unicorn_stream[0])

fs = 250.0
 
channels = ["Fz","C3", "Cz", "C4","Pz","PO7","Oz","PO8"]

info = mne.create_info(
    ch_names= channels,
    ch_types= ['eeg']*len(channels),
    sfreq= fs
)

csv_file = 'real_time_data.csv'

if os.path.exists(csv_file):
    open(csv_file, 'w').close()

while True:
    sample, timestamp = inlet.pull_sample()
    # print(sample)

    if sample:
        data1 = sample[0:8]
        data.append(data1)  

        time_count += 1 * 0.004
        time_count = np.round(time_count ,3)
        time_all += 1 * 0.004

        data = np.asarray(data)
        epoch_2s = np.delete(epoch_2s, slice(0,1), axis =1)
        epoch_2s = np.append(epoch_2s, data.T, axis= 1)

        data = np.ndarray.tolist(data)
        data = []

        if time_count == time_interval:

            raw = mne.io.RawArray(epoch_2s, info, verbose=False)
            eeg1= raw.pick(["Fz","C3", "Cz", "C4","Pz",'PO7','Oz','PO8'])
            mne_epoch = mne.EpochsArray(eeg1.get_data().reshape(1,8,751), info, baseline=(0.0,0.5), verbose=False)
            mne_epoch = mne_epoch.copy().crop(tmin=0.5, tmax=2.5)

            new_data2 = trained_csp2.transform(mne_epoch.get_data().reshape(1,8,501))

            if time_all > 1.0:

                new_data2 = trained_csp2.transform(mne_epoch.get_data().reshape(1,8,501))
                print(trained_lda.predict(new_data2), datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
                
                new_data_point = {
                'Timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'Class_Raw': trained_lda.predict_proba(new_data2),
                'Class': trained_lda.predict(new_data2)[0]
                }
                # Create a DataFrame from the new_data_point
                df = pd.DataFrame([new_data_point])
                try:
                    df.to_csv(csv_file, mode='x', index=False)
                except FileExistsError:
                    df.to_csv(csv_file, mode='a', header=False, index=False)
        
            time_count = 0.0

        # if time_all >= 200.0:
        #     break


looking for an EEG stream...
[<pylsl.pylsl.StreamInfo object at 0x000001E0F37FCA10>, <pylsl.pylsl.StreamInfo object at 0x000001E0F37FC190>]
openvibefilter
openvibeMarker


C:\Users\pipo_\AppData\Roaming\Python\Python311\site-packages\mne\decoding\csp.py:241: RuntimeWarning: divide by zero encountered in log
  X = np.log(X)
C:\Users\pipo_\AppData\Roaming\Python\Python311\site-packages\mne\decoding\csp.py:241: RuntimeWarning: divide by zero encountered in log
  X = np.log(X)
C:\Users\pipo_\AppData\Roaming\Python\Python311\site-packages\mne\decoding\csp.py:241: RuntimeWarning: divide by zero encountered in log
  X = np.log(X)
C:\Users\pipo_\AppData\Roaming\Python\Python311\site-packages\mne\decoding\csp.py:241: RuntimeWarning: divide by zero encountered in log
  X = np.log(X)
C:\Users\pipo_\AppData\Roaming\Python\Python311\site-packages\mne\decoding\csp.py:241: RuntimeWarning: divide by zero encountered in log
  X = np.log(X)


[0] 2024-01-20 18:22:58
[0] 2024-01-20 18:22:58
[0] 2024-01-20 18:22:58
[0] 2024-01-20 18:22:58
[0] 2024-01-20 18:22:58
[0] 2024-01-20 18:22:58
[0] 2024-01-20 18:22:58
[0] 2024-01-20 18:22:58
[0] 2024-01-20 18:22:59
[0] 2024-01-20 18:22:59
[0] 2024-01-20 18:22:59
[0] 2024-01-20 18:22:59
[0] 2024-01-20 18:22:59
[0] 2024-01-20 18:22:59
[0] 2024-01-20 18:22:59
[1] 2024-01-20 18:22:59
[1] 2024-01-20 18:22:59
[1] 2024-01-20 18:22:59
[1] 2024-01-20 18:22:59
[1] 2024-01-20 18:23:00
[1] 2024-01-20 18:23:00
[1] 2024-01-20 18:23:00
[1] 2024-01-20 18:23:00
[1] 2024-01-20 18:23:00
[1] 2024-01-20 18:23:00
[1] 2024-01-20 18:23:00
[1] 2024-01-20 18:23:00
[1] 2024-01-20 18:23:00
[1] 2024-01-20 18:23:00
[1] 2024-01-20 18:23:01
[1] 2024-01-20 18:23:01
[1] 2024-01-20 18:23:01
[1] 2024-01-20 18:23:01
[1] 2024-01-20 18:23:01
[1] 2024-01-20 18:23:01
[1] 2024-01-20 18:23:01
[1] 2024-01-20 18:23:01
[1] 2024-01-20 18:23:01
[1] 2024-01-20 18:23:01
[1] 2024-01-20 18:23:02
[1] 2024-01-20 18:23:02
[1] 2024-01-20 1

KeyboardInterrupt: 